# BMGT347 Final Group Project

In [110]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures, add_dummy_feature
from sklearn.ensemble import RandomForestRegressor
import statsmodels.formula.api as smf
import sklearn.linear_model as skl_lm
import statsmodels.api as sm
from sklearn.preprocessing import scale

In [111]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [112]:
#upload the file to your Google Drive and change the path accordingly
#for example, if you upload the csv file to your Google Drive folder directly, then you can run: data = pd.read_csv('/content/drive/My Drive/characteristics_anom.csv')

data = pd.read_csv('/content/drive/My Drive/characteristics_anom.csv')
data.head()
print(data.columns)

Index(['permno', 'date', 're', 'size', 'value', 'prof', 'valprof', 'fscore',
       'debtiss', 'repurch', 'nissa', 'accruals', 'growth', 'aturnover',
       'gmargins', 'ep', 'cfp', 'noa', 'inv', 'invcap', 'igrowth', 'sgrowth',
       'lev', 'roaa', 'roea', 'sp', 'mom', 'indmom', 'valmom', 'valmomprof',
       'mom12', 'momrev', 'lrrev', 'valuem', 'nissm', 'roe', 'rome', 'strev',
       'ivol', 'betaarb', 'season', 'indrrev', 'ciss', 'price', 'age',
       'shvol'],
      dtype='object')


In [113]:
#create a new variable for year
data['year'] = pd.DatetimeIndex(data['date']).year

In [114]:
data = data.set_index(['permno', 'date'])
X_var_names = data.columns[1:]
y_var_name = data.columns[0]
print('X variables: ', X_var_names.values)
print('y variable: ', y_var_name)

X variables:  ['size' 'value' 'prof' 'valprof' 'fscore' 'debtiss' 'repurch' 'nissa'
 'accruals' 'growth' 'aturnover' 'gmargins' 'ep' 'cfp' 'noa' 'inv'
 'invcap' 'igrowth' 'sgrowth' 'lev' 'roaa' 'roea' 'sp' 'mom' 'indmom'
 'valmom' 'valmomprof' 'mom12' 'momrev' 'lrrev' 'valuem' 'nissm' 'roe'
 'rome' 'strev' 'ivol' 'betaarb' 'season' 'indrrev' 'ciss' 'price' 'age'
 'shvol' 'year']
y variable:  re


In [115]:
data[X_var_names]

,,size,value,prof,valprof,fscore,debtiss,repurch,nissa,accruals,growth,...,strev,ivol,betaarb,season,indrrev,ciss,price,age,shvol,year
permno,date,,,,,,,,,,,,,,,,,,,,,
10006,07/1972,0.699925,0.649009,-0.773268,-0.441137,-1.057577,-1.191008,-0.443104,-2.053981,0.163833,-0.265807,...,0.169149,-0.253520,-0.597934,-0.762987,-0.549901,-1.220848,1.117585,1.213013,0.524622,1972
10102,07/1972,0.538023,1.406381,-0.604513,0.939781,0.432276,-1.191008,-0.443104,0.827987,0.266576,-0.647289,...,-0.677317,-0.658350,0.556359,-1.199386,-0.482659,-0.027322,-0.683487,1.213013,-0.411568,1972
10145,07/1972,1.379123,1.073137,-0.641500,0.334379,0.432276,-1.191008,-0.443104,-0.480636,-0.022215,-0.622677,...,-0.095248,0.235483,0.346043,0.557622,-0.108870,-0.970639,0.395970,1.213013,0.102147,1972
10153,07/1972,0.186578,1.688356,-0.463498,1.385077,-0.339504,-1.191008,2.255521,-2.121410,-0.502607,-0.876178,...,-1.223870,0.596222,0.776458,0.432121,-1.216393,0.660034,-1.280549,1.213013,0.177518,1972
10161,07/1972,1.313967,0.667652,-1.170883,-0.898941,-1.057577,-1.191008,-0.443104,-0.195935,0.463731,1.070611,...,0.944583,-1.145348,-0.350934,-0.680270,0.721773,-0.579507,0.427603,1.213013,-0.645615,1972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93420,12/2017,-0.412146,1.475956,-1.091624,0.187793,-1.211798,-0.764533,0.776055,1.583291,-0.535687,0.559501,...,0.894187,1.461344,1.722207,-1.695544,1.031494,1.679213,-1.609926,-0.864585,1.629615,2017
93423,12/2017,0.589454,-1.512324,0.263496,-1.511108,-0.461570,-0.764533,0.776055,-1.259711,0.052951,-0.387347,...,0.413537,-0.003766,-1.423774,0.900504,0.464172,-1.500375,0.609028,-0.864585,1.317163,2017
93428,12/2017,-0.842212,-0.805159,0.961403,0.187793,1.369496,1.307284,-1.287875,1.068881,1.355837,0.623558,...,-0.687486,-0.949121,0.520185,0.779973,0.061125,0.281030,0.323973,-0.864585,1.655809,2017


# Preventing multi-collinearity

In [116]:
correlation_matrix = data.drop('re', axis=1).corr() # correlation matrix with correlation coefficients for features

correlation_matrix = correlation_matrix.abs()

highest_corr = (correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool)) # create a list of feature pairs
                 .stack()
                 .sort_values(ascending=False))

top_corr_pairs = highest_corr.head(10) # print top 10 pairs of features that are most highly correlated
print(top_corr_pairs)

strev  indrrev       0.895377
value  valuem        0.862420
       lev           0.854197
roaa   roea          0.849366
size   noa           0.768121
lev    valuem        0.744991
mom    valmomprof    0.742990
       valmom        0.742149
ep     cfp           0.735346
cfp    lev           0.735158
dtype: float64


In [119]:
df = data.drop('re', axis=1).copy()

# Counting the occurences of each feature in the top 10 to determine what variables have more correlation
appearance_count = {}

for (var1, var2) in top_corr_pairs.index:
    if var1 in appearance_count:
        appearance_count[var1] += 1
    else:
        appearance_count[var1] = 1

    if var2 in appearance_count:
        appearance_count[var2] += 1
    else:
        appearance_count[var2] = 1

print(appearance_count)
variables_to_remove = ['lev', 'valuem','cfp' ] # remove these as they have high counts and are logically redundant
processed_data = df.drop(variables_to_remove, axis=1)
processed_data['re'] = data['re']

print(processed_data.shape)

{'strev': 1, 'indrrev': 1, 'value': 2, 'valuem': 2, 'lev': 3, 'roaa': 1, 'roea': 1, 'size': 1, 'noa': 1, 'mom': 2, 'valmomprof': 1, 'valmom': 1, 'ep': 1, 'cfp': 2}
(502636, 42)


processed_data now contains the appropriate variables with 3 removed to prevent multicollinearity within our regression model

In [120]:
train = processed_data.loc[data['year']<2005]
test = processed_data.loc[data['year']>=2005]

# Model Selection with Cross Validation

In [121]:
coll_drop = ['re', 'price', 'indrrev'] # drop these as they are either return or derivative of other features
X_train = train.drop(coll_drop, axis=1)
X_test = test.drop(coll_drop, axis=1)
y_train = train.re
y_test = test.re

In [122]:
regr = skl_lm.LinearRegression()
regr.fit(scale(X_train), y_train) # fit to train data

mse_scores = cross_val_score(regr, scale(X_train), y_train, cv=5) # runs cross validation on train set with 5 folds
mean_mse = np.mean(mse_scores)
print("MSE: ", mean_mse)


MSE:  0.0019119820193546078


In [123]:
print(regr.coef_)

[-1.59218304e-03 -1.00848100e-03 -5.80149144e-05 -2.37346772e-04
  1.05756574e-04  5.59631996e-04 -1.31791481e-04  6.72293039e-04
 -4.69308910e-04  1.19297286e-04 -1.59901792e-03  1.92388279e-05
 -1.24895901e-04  1.23404258e-03 -7.19103444e-04  6.54156296e-04
 -3.48472948e-04  1.90013322e-04 -1.28398108e-03  7.74241488e-04
  1.17454607e-03 -5.00078450e-03  1.81852598e-03  9.61865226e-04
  2.87534500e-03  2.38550996e-03 -1.64085056e-03 -1.09607505e-03
 -1.45304180e-03  6.71941885e-04  3.14315923e-03 -4.81839000e-03
  8.42910492e-04  1.12558916e-03  3.03623964e-03 -1.10410976e-03
 -1.97594000e-03 -2.51805871e-04  1.33285362e-03]


In [124]:
alphas = [.005,.01,.05, .1]

In [125]:
ridgecv = RidgeCV(alphas=alphas) # cross validation with the alphas(hyperparameter) to pick best one
ridgecv.fit(scale(X_train), y_train) # fit to train data

mse_scores = cross_val_score(ridgecv, scale(X_train), y_train, cv=5) # runs cross validation on train set with 5 folds
mean_mse = np.mean(abs(mse_scores))
print("MSE: ", mean_mse)
print("Alpha: ", ridgecv.alpha_)
# 2 mins to run

MSE:  0.001911990338743852
Alpha:  0.1


In [126]:
print(ridgecv.coef_) # coefficients of features(betas)

[-1.59217705e-03 -1.00844812e-03 -5.80068182e-05 -2.37329952e-04
  1.05756479e-04  5.59631207e-04 -1.31791378e-04  6.72291997e-04
 -4.69308686e-04  1.19296862e-04 -1.59901243e-03  1.92419353e-05
 -1.24894317e-04  1.23403606e-03 -7.19102842e-04  6.54155227e-04
 -3.48472625e-04  1.90013414e-04 -1.28398175e-03  7.74239846e-04
  1.17454431e-03 -5.00072921e-03  1.81852444e-03  9.61834979e-04
  2.87530162e-03  2.38550561e-03 -1.64085026e-03 -1.09607449e-03
 -1.45304044e-03  6.71942967e-04  3.14315721e-03 -4.81838830e-03
  8.42909825e-04  1.12558887e-03  3.03623921e-03 -1.10410912e-03
 -1.97593898e-03 -2.51806099e-04  1.33285376e-03]


In [127]:
lassocv = LassoCV(alphas=alphas)# cross validation with the alphas(hyperparameter) to pick best one
lassocv.fit(scale(X_train), y_train) # fit to train data

mse_scores = cross_val_score(lassocv, scale(X_train), y_train, cv=5)# runs cross validation on train set with 5 folds
mean_mse = np.mean(abs(mse_scores))
print("MSE: ", mean_mse)
print("Alpha: ", lassocv.alpha_)


MSE:  0.0010723204090733684
Alpha:  0.1


In [128]:
print(lassocv.coef_) # coefficients of features(betas)

[-0.  0.  0.  0.  0.  0.  0. -0. -0. -0.  0. -0.  0. -0. -0. -0. -0. -0.
 -0. -0.  0.  0.  0.  0.  0.  0. -0. -0. -0.  0.  0. -0.  0.  0.  0. -0.
 -0.  0.  0.]


In [129]:
ridge_coefficients = ridgecv.coef_
feature_names = X_train.columns

coef_abs_tuples = [(feature_name, abs(coef)) for feature_name, coef in zip(feature_names, ridge_coefficients)] # coefficients of features(betas) list

sorted_coef_abs_tuples = sorted(coef_abs_tuples, key=lambda x: x[1], reverse=True) # absolute values of the coefficients

# Print sorted feature coefficients
print("Sorted Coefficients:")
for feature_name, coef_abs in sorted_coef_abs_tuples:
    print(f"{feature_name}: {coef_abs}")

Sorted Coefficients:
mom: 0.00500072921075656
strev: 0.004818388299717924
rome: 0.0031431572057130097
season: 0.0030362392123421955
valmomprof: 0.0028753016201363835
mom12: 0.002385505613631267
age: 0.0019759389817295414
indmom: 0.0018185244434789638
momrev: 0.0016408502625182564
aturnover: 0.0015990124308729037
size: 0.0015921770466373886
nissm: 0.0014530404359476279
year: 0.0013328537564234466
roaa: 0.001283981753473995
noa: 0.0012340360648508408
sp: 0.0011745443101514041
betaarb: 0.0011255888728101127
ciss: 0.0011041091187209418
lrrev: 0.0010960744886450158
value: 0.0010084481221923625
valmom: 0.0009618349789803915
ivol: 0.0008429098249820299
roea: 0.0007742398460948152
inv: 0.0007191028421686951
nissa: 0.0006722919974073704
roe: 0.0006719429668393673
invcap: 0.0006541552269965223
debtiss: 0.0005596312068192333
accruals: 0.0004693086864762108
igrowth: 0.00034847262460413475
shvol: 0.0002518060988165871
valprof: 0.00023732995154190917
sgrowth: 0.00019001341428293017
repurch: 0.000131

# **Final Model Evaluation**

In [130]:
mse_scores = cross_val_score(ridgecv, scale(X_test), y_test, cv=5) # cross validation on the test data
mean_mse = np.mean(abs(mse_scores))
print("MSE: ", mean_mse)
print("Alpha: ", ridgecv.alpha_)


MSE:  0.007008092343424544
Alpha:  0.1


In [131]:
from sklearn.metrics import r2_score

r2_scores = cross_val_score(ridgecv, scale(X_test), y_test, cv=5, scoring='r2') # CV to determine r2
mean_r2 = np.mean(r2_scores)
print("R^2 Score: ", mean_r2)

R^2 Score:  -0.007008092343424544
